This tutorial assumes the reader is already familiar with `tcpdump`, the command-line packet analyzer. Given a `tcpdump` command, we will show how you can find the equivalent set of network packets with the Intake PCAP plugin.

### Intake Catalog

The catalog examples below are defined with the following format (see `examples/sample.yml`):

```
plugins:
  source:
    - module: intake_pcap
sources:
  - name: raw_live
    driver: pcap
    args:
      urlpath: ~
      interface: en0
      chunksize: 10
  - name: raw_local
    driver: pcap
    args:
      urlpath: !template '{{ CATALOG_DIR }}/local.pcap'
  - name: tcp_local
    driver: pcap
    args:
      urlpath: !template '{{ CATALOG_DIR }}/local.pcap'
      protocol: tcp
  - name: udp_local
    driver: pcap
    args:
      urlpath: !template '{{ CATALOG_DIR }}/local.pcap'
      protocol: udp
```

### Preparation

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 1000)

These examples require that you run this command, `sudo tcpdump -i en0 -c 10 -w local.pcap`, in order to create the local PCAP data.

These examples also assume the default network interface is `en0`, which is used on macOS. See your local documentation for the default network interface if your OS is different.

### Read live stream

To read a live stream of packets, you need to start Jupyter using `sudo jupyter notebook --allow-root`.

**NOTE**: Intake does not currently support streaming packets from the network interface. Packets will be placed into a dataframe in chunks (which can be adjusted by the user).

#### Example: tcpdump

This example will show the first 10 packets on the interface `en0`. Each packet will be timestamped and the raw IP address will be displayed. No packets will be filtered.

`sudo tcpdump -i en0 -c 10 -tttt -n -q`

```
2018-01-08 23:37:21.882212 IP 8.8.8.8.53 > 192.168.0.39.61362: UDP, length 172
2018-01-08 23:37:21.882927 IP 192.168.0.39.61447 > 52.12.34.56.443: tcp 0
2018-01-08 23:37:21.953415 IP 52.23.45.67.443 > 192.168.0.39.61445: tcp 0
2018-01-08 23:37:21.953528 IP 192.168.0.39.61445 > 52.23.45.67.443: tcp 0
2018-01-08 23:37:21.991435 IP 52.12.34.56.443 > 192.168.0.39.61447: tcp 0
2018-01-08 23:37:21.991523 IP 192.168.0.39.61447 > 52.12.34.56.443: tcp 0
2018-01-08 23:37:21.993620 IP 192.168.0.39.61447 > 52.12.34.56.443: tcp 517
2018-01-08 23:37:22.093955 IP 52.12.34.56.443 > 192.168.0.39.61447: tcp 0
2018-01-08 23:37:22.099580 IP 52.12.34.56.443 > 192.168.0.39.61447: tcp 1448
2018-01-08 23:37:22.099587 IP 52.12.34.56.443 > 192.168.0.39.61447: tcp 1448
```

#### Example: Get stream of packets without catalog

This example is equivalent to the `tcpdump` example, except the packets will be available in a dataframe.

In [ ]:
from intake_pcap import LiveStream
lstream = LiveStream("en0")
df0 = lstream.to_dataframe(n=10)
df0

```
                       time       src_host src_port         dst_host dst_port protocol
0 2018-01-09 07:42:36.055605   52.12.34.56      443     192.168.0.39    61614      tcp
1 2018-01-09 07:42:36.055682  192.168.0.39    61614      52.12.34.56      443      tcp
2 2018-01-09 07:42:37.839555  192.168.0.39    17500  255.255.255.255    17500      udp
3 2018-01-09 07:42:37.840472  192.168.0.39    17500    192.168.0.255    17500      udp
4 2018-01-09 07:42:37.890092  192.168.0.39    61614      52.12.34.56      443      tcp
5 2018-01-09 07:42:37.890243  192.168.0.39    61616      52.12.34.56      443      tcp
6 2018-01-09 07:42:37.912166   52.12.34.56      443     192.168.0.39    61616      tcp
7 2018-01-09 07:42:37.912237  192.168.0.39    61616      52.12.34.56      443      tcp
8 2018-01-09 07:42:37.912399  192.168.0.39    61616      52.12.34.56      443      tcp
9 2018-01-09 07:42:37.912833  192.168.0.39    61376     104.12.34.56     4070      tcp
```

#### Example: Get stream of packets with catalog

This example is equivalent to the `tcpdump` example, except the packets will be available in a dataframe.

In [ ]:
from intake.catalog import Catalog
c = Catalog("sample.yml")
src = c["raw_live"].get()
df1 = src.read()
df1

```
                        time      src_host src_port         dst_host dst_port protocol
0 2018-01-09 07:47:26.825023  192.168.0.1    36123  239.255.255.250     1900      udp
1 2018-01-09 07:47:26.825845  192.168.0.1    36123  239.255.255.250     1900      udp
2 2018-01-09 07:47:26.826602  192.168.0.1    36123  239.255.255.250     1900      udp
3 2018-01-09 07:47:26.827547  192.168.0.1    36123  239.255.255.250     1900      udp
4 2018-01-09 07:47:26.828168  192.168.0.1    36123  239.255.255.250     1900      udp
5 2018-01-09 07:47:26.829162  192.168.0.1    36123  239.255.255.250     1900      udp
6 2018-01-09 07:47:26.829865  192.168.0.1    36123  239.255.255.250     1900      udp
7 2018-01-09 07:47:26.830832  192.168.0.1    36123  239.255.255.250     1900      udp
8 2018-01-09 07:47:26.831615  192.168.0.1    36123  239.255.255.250     1900      udp
9 2018-01-09 07:47:26.832476  192.168.0.1    36123  239.255.255.250     1900      udp
```

### Read PCAP file

#### Example: tcpdump

This example will show the first 10 packets from `local.pcap`. Each packet will be timestamped and the raw IP address will be displayed. No packets will be filtered.

`sudo tcpdump -c 10 -tttt -n -q -r local.pcap`

```
2018-01-09 00:16:12.210010 IP 192.168.0.39.54703 > 172.123.4.567.443: UDP, length 1350
2018-01-09 00:16:12.210910 IP 192.168.0.39.54703 > 172.123.4.567.443: UDP, length 998
2018-01-09 00:16:12.236176 IP 172.123.4.567.443 > 192.168.0.39.54703: UDP, length 1350
2018-01-09 00:16:12.236543 IP 172.123.4.567.443 > 192.168.0.39.54703: UDP, length 31
2018-01-09 00:16:12.236726 IP 192.168.0.39.54703 > 172.123.4.567.443: UDP, length 41
2018-01-09 00:16:12.236791 IP 192.168.0.39.54703 > 172.123.4.567.443: UDP, length 38
2018-01-09 00:16:12.251367 STP 802.1d, Config, Flags [none], bridge-id 7b00.01:23:45:67:89:00.8002, length 35
2018-01-09 00:16:12.252565 IP 172.123.4.567.443 > 192.168.0.39.54703: UDP, length 30
2018-01-09 00:16:12.313082 IP 172.123.4.567.443 > 192.168.0.39.54703: UDP, length 814
2018-01-09 00:16:12.313479 IP 172.123.4.567.443 > 192.168.0.39.54703: UDP, length 16
```

This example is equivalent to the `tcpdump` example, except the packets will be available in a dataframe.

#### Example: Get stream of packets without catalog

In [ ]:
from intake_pcap import OfflineStream
ostream = OfflineStream("local.pcap")
ostream.set_filter("udp")
df2 = ostream.to_dataframe()
df2

```
                        time       src_host src_port       dst_host dst_port protocol
0 2018-01-09 08:16:12.210010   192.168.0.39    54703  172.123.4.567      443      udp
1 2018-01-09 08:16:12.210910   192.168.0.39    54703  172.123.4.567      443      udp
2 2018-01-09 08:16:12.236176  172.123.4.567      443   192.168.0.39    54703      udp
3 2018-01-09 08:16:12.236543  172.123.4.567      443   192.168.0.39    54703      udp
4 2018-01-09 08:16:12.236726   192.168.0.39    54703  172.123.4.567      443      udp
5 2018-01-09 08:16:12.236791   192.168.0.39    54703  172.123.4.567      443      udp
6 2018-01-09 08:16:12.252565  172.123.4.567      443   192.168.0.39    54703      udp
7 2018-01-09 08:16:12.313082  172.123.4.567      443   192.168.0.39    54703      udp
8 2018-01-09 08:16:12.313479  172.123.4.567      443   192.168.0.39    54703      udp
```

This example is equivalent to the `tcpdump` example, except the packets will be available in a dataframe.

#### Example: Get stream of packets with catalog

In [ ]:
from intake.catalog import Catalog
c = Catalog("sample.yml")
src = c["udp_local"].get()
df3 = src.read()
df3

```
                        time       src_host src_port       dst_host dst_port protocol
0 2018-01-09 08:16:12.210010   192.168.0.39    54703  172.123.4.567      443      udp
1 2018-01-09 08:16:12.210910   192.168.0.39    54703  172.123.4.567      443      udp
2 2018-01-09 08:16:12.236176  172.123.4.567      443   192.168.0.39    54703      udp
3 2018-01-09 08:16:12.236543  172.123.4.567      443   192.168.0.39    54703      udp
4 2018-01-09 08:16:12.236726   192.168.0.39    54703  172.123.4.567      443      udp
5 2018-01-09 08:16:12.236791   192.168.0.39    54703  172.123.4.567      443      udp
6 2018-01-09 08:16:12.252565  172.123.4.567      443   192.168.0.39    54703      udp
7 2018-01-09 08:16:12.313082  172.123.4.567      443   192.168.0.39    54703      udp
8 2018-01-09 08:16:12.313479  172.123.4.567      443   192.168.0.39    54703      udp
```